# Content Based Recommender System

# Informasi Dataset

Proyek ini menggunakan kumpulan data MovieLens, kumpulan data yang banyak digunakan di bidang sistem pemberi rekomendasi, yang berisi data film dan metadata.

terdapat 2 file :
movies.csv 
ratings.csv

Link dataset : [Dataset](https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system/data)

# Load Dataset

Pada kode berikut untuk memuat dataset melalui platform kaggle notebook.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


# Read Dataset

Kode berikut untuk memuat dataset (.csv)

In [2]:
data_movies = pd.read_csv("/kaggle/input/movie-recommendation-system/movies.csv")
data_ratings = pd.read_csv("/kaggle/input/movie-recommendation-system/ratings.csv")

Kode berikut untuk melihat contoh data pada file movies.csv, dapat dilihat terdapat 3 kolom pada dataframe ini : `movieId`, `title`, `genres`.

In [3]:
data_movies.head(-10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62408,209135,Jane B. by Agnès V. (1988),Documentary|Fantasy
62409,209137,The Reward's Yours... The Man's Mine (1969),Western
62410,209139,Rimsky-Korsakov (1953),Drama
62411,209141,And They Lived Happily Ever After (1976),Comedy


# Data Preprocessing

Kode berikut untuk melihat ringkasan terkait dataframe movies, dapat dilihat tipe data untuk tiap kolom beserta jumlah baris dataframe movies yaitu sebanyak **62,423**.

In [5]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


Kode berikut untuk melihat ringkasan terkait dataframe ratings, dapat dilihat tipe data untuk tiap kolom beserta jumlah baris dataframe ratings yaitu sebanyak **25,000,095**.

In [6]:
data_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


selanjutnya mengecek apakah dataframe movies ada data yang kosong (null), dapat terlihat bahwa setiap kolom tidak ada data yang kosong.

In [7]:
data_movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

sama seperti sebelumnya selanjutnya mengecek apakah dataframe ratings ada data yang kosong (null), dapat terlihat bahwa setiap kolom tidak ada data yang kosong.

In [8]:
data_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

selanjutnya kita akan mengecek berapa jumlah data movieId yang unik pada kedua dataframe, terdapat selisih antara kedua data tersebut, kita asumsikan bahwa dataframe ratings berisi data film yang telah ditonton oleh user.

In [9]:
print(f"Jumlah jenis Movies di data Movies  : {data_movies['movieId'].nunique()}")
print(f"Jumlah jenis Movies di data Ratings : {data_ratings['movieId'].nunique()}")

Jumlah jenis Movies di data Movies  : 62423
Jumlah jenis Movies di data Ratings : 59047


# Data Exploration

Kita akan melakukan eksplorasi data untuk mengetahui manakah film yang paling banyak ditonton/diberi rating oleh user. pertama kita akan menghitung data unik yang muncul pada dataframe ratings, kemudian ambil 50 data pertama ke dalam sebuah list. Pada kode berikut untuk mengambil data `movieId`.

In [10]:
arr_mov_most =  data_ratings['movieId'].value_counts()[:50].index
arr_count_most = data_ratings['movieId'].value_counts()[:50].values
arr_mov_most


Index([  356,   318,   296,   593,  2571,   260,   480,   527,   110,  2959,
         589,  1196,     1,  4993,    50,  1210,  1198,  2858,   858,  5952,
        7153,    47,   457,  1270,   780,   150,   608,    32,  2028,  2762,
        3578,   592,   588,   364,  4306,   380,   590, 58559,   377,  4226,
        1580,  1704, 79132,  1240,  1291,  1197,  1136,  1721,  1265,   344],
      dtype='int64', name='movieId')

kemudian dari list top 50 data movieId tadi kita gunakan untuk mencari data title ke dalam list.

In [11]:
arr_title_most = [data_movies.loc[data_movies['movieId']==x]['title'].values[0] for x in arr_mov_most]

Tampilkan data hasil eksplorasi tersebut ke dalam bentuk DataFrame, dapat kita lihat bahwa film `Forrest Gump (1994)` menjadi film yang paling banyak ditonton/dirating oleh user disusul oleh `Shawshank Redemption, The (1994)`.

In [12]:
df = pd.DataFrame({
    "movieId" : arr_mov_most, 
    "title" : arr_title_most, 
    "jumlah" : arr_count_most
})

df.head(5)

,movieId,title,jumlah
0,356,Forrest Gump (1994),81491
1,318,"Shawshank Redemption, The (1994)",81482
2,296,Pulp Fiction (1994),79672
3,593,"Silence of the Lambs, The (1991)",74127
4,2571,"Matrix, The (1999)",72674


# Data Preparation

Selanjutnya kita akan mengubah data pada kolom `genres` menjadi huruf kecil, untuk menjaga konsistensi data.

In [17]:
data_movies['genres'] = data_movies['genres'].apply(lambda x: x.lower())

Pada proyek ini akan menggunakan teknik embedding menggunakan FastText kemudian mengunakan Cosien Similariy untuk perhitungan kedekatan antar data, untuk data yang kita embedding adalah data `title` dan `genres`.

Maka dari itu pada kode ini kita akan menggabungkan kedua data pada tiap baris tersebut menjadi satu string, sebelumnya kita hilangkan tanda `|` pada data genres, kemudian akan menyimpan hasil proses tersebut ke kolom `text`.

In [20]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity

data_movies["text"] = data_movies["title"] + " " + data_movies["genres"].str.replace("|", " ")
data_movies.head()

,movieId,title,genres,text
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,Toy Story (1995) adventure animation children ...
1,2,Jumanji (1995),adventure|children|fantasy,Jumanji (1995) adventure children fantasy
2,3,Grumpier Old Men (1995),comedy|romance,Grumpier Old Men (1995) comedy romance
3,4,Waiting to Exhale (1995),comedy|drama|romance,Waiting to Exhale (1995) comedy drama romance
4,5,Father of the Bride Part II (1995),comedy,Father of the Bride Part II (1995) comedy


Selanjutnya kita menerepakan tokenisasi setiap data pada kolom `text` pada dataframe.

In [21]:
words_token = [str(x).split() for x in data_movies['text']]
words_token[:2]

[['Toy',
  'Story',
  '(1995)',
  'adventure',
  'animation',
  'children',
  'comedy',
  'fantasy'],
 ['Jumanji', '(1995)', 'adventure', 'children', 'fantasy']]

# Training Model Embedding

selanjutnya kita akan melatih model FastText ke data text yang telah kita pecah menjadi token-token, atur agar ukuran vector 30, dengan epochs 10.

In [22]:
model = FastText(words_token, vector_size=30, window=3, min_count=1, epochs=10)

Pada kode berikut adalah fungsi untuk mendapatkan data embedding dari hasil training model FastText, jadi nantinya setiap data pada kolom `text` tersebut akan diubah menjadi bentuk vector embedding dengan fungsi ini.

In [23]:
def get_embedding(text):
    word = str(text).split()
    vector = [model.wv[x] for x in word if x in model.wv]
    if vector:
        return sum(vector) / len(vector)
    else:
        return np.zeros(vector.vector_size)

Apply ke setiap kolom `text` untuk mengubah data tersebut menjadi bentuk vector dan simpan ke kolom `embedding`.

In [24]:
data_movies["embedding"] = data_movies['text'].apply(get_embedding)
data_movies.head()

,movieId,title,genres,text,embedding
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,Toy Story (1995) adventure animation children ...,"[0.0031403974, 0.08475217, 0.21156888, 0.09668..."
1,2,Jumanji (1995),adventure|children|fantasy,Jumanji (1995) adventure children fantasy,"[0.15555722, -0.48341304, -0.1914331, 0.461303..."
2,3,Grumpier Old Men (1995),comedy|romance,Grumpier Old Men (1995) comedy romance,"[-0.7260923, 0.20993322, -0.09047246, -0.15078..."
3,4,Waiting to Exhale (1995),comedy|drama|romance,Waiting to Exhale (1995) comedy drama romance,"[-1.833497, 0.68149275, -0.508115, -0.3380818,..."
4,5,Father of the Bride Part II (1995),comedy,Father of the Bride Part II (1995) comedy,"[-0.88400465, 0.9350853, 1.4404155, -1.1431693..."


contoh hasil data embedding

In [25]:
data_movies["embedding"][0]

array([ 3.14039737e-03,  8.47521722e-02,  2.11568877e-01,  9.66898650e-02,
       -6.64973021e-01, -4.26654279e-01,  4.34927344e-01,  1.35666418e+00,
       -3.55884838e+00,  2.36385778e-01,  8.71300697e-02, -2.20618081e+00,
        2.33100557e+00, -1.40390062e+00,  3.49373966e-01, -1.22287594e-01,
       -8.05963397e-01,  2.23506331e+00, -1.26600361e+00, -2.00376225e+00,
        4.66963798e-01,  8.67755532e-01, -3.44223380e-01, -2.19663143e-01,
       -6.88504994e-01,  6.79856300e-01, -1.56165993e+00, -5.64303160e-01,
        2.01109618e-01,  2.41778821e-01], dtype=float32)

# Create Similarity Matrix

Selanjutnya data embedding tersebut kita lakukan stack secara vertikal, kemudian membuat `similarity matrix` dari array embedding tersebut. 

In [26]:
embedd = np.vstack(data_movies["embedding"].to_numpy())
similarity_matrix = cosine_similarity(embedd)

contoh similarity matrix

In [27]:
similarity_matrix[:2]

array([[1.        , 0.963272  , 0.86567616, ..., 0.8798458 , 0.61943066,
        0.8425486 ],
       [0.963272  , 1.0000001 , 0.8082077 , ..., 0.81610376, 0.45492983,
        0.7866636 ]], dtype=float32)

agar index pada dataframe berurutan mulai dari index ke-0, menggunakan kode ini.

In [28]:
data_movies = data_movies.reset_index(drop=True)
data_movies.head(-1)


,movieId,title,genres,text,embedding
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,Toy Story (1995) adventure animation children ...,"[0.0031403974, 0.08475217, 0.21156888, 0.09668..."
1,2,Jumanji (1995),adventure|children|fantasy,Jumanji (1995) adventure children fantasy,"[0.15555722, -0.48341304, -0.1914331, 0.461303..."
2,3,Grumpier Old Men (1995),comedy|romance,Grumpier Old Men (1995) comedy romance,"[-0.7260923, 0.20993322, -0.09047246, -0.15078..."
3,4,Waiting to Exhale (1995),comedy|drama|romance,Waiting to Exhale (1995) comedy drama romance,"[-1.833497, 0.68149275, -0.508115, -0.3380818,..."
4,5,Father of the Bride Part II (1995),comedy,Father of the Bride Part II (1995) comedy,"[-0.88400465, 0.9350853, 1.4404155, -1.1431693..."
...,...,...,...,...,...
62417,209155,Santosh Subramaniam (2008),action|comedy|romance,Santosh Subramaniam (2008) action comedy romance,"[0.17410927, 0.1486553, -0.62004346, 0.3366032..."
62418,209157,We (2018),drama,We (2018) drama,"[-1.8060676, 0.63919, -0.41909924, 0.40797624,..."
62419,209159,Window of the Soul (2001),documentary,Window of the Soul (2001) documentary,"[-0.6736467, 0.9119425, 1.7143146, -0.97022027..."
62420,209163,Bad Poems (2018),comedy|drama,Bad Poems (2018) comedy drama,"[-0.7083064, 0.25430274, -0.28544492, 0.223032..."


# Testing Data

Terakhir, kita akan menguji coba dengan step:

- Input nama title film
- Mencari movieId yang sesuai dengan title
- Mendapatkan index data movieId tersebut
- Mencari 10 data dari similarity matrix yang paling mirip dengan data input
- Tampilkan data tersebut ke DataFrame

Hasil pengujian menunjukkan metode dengan embedding menggunakan model FastText dan pencarian cosine similarity memiliki hasil yang cukup baik dalam memberikan rekomendasi film yang mirip secara genre.

In [40]:
nama = input("")
movie_id = data_movies[data_movies['title']==nama]['movieId'].values[0]
movie_data = data_movies[data_movies['movieId']==movie_id]["movieId"]
film_index = movie_data.values[0]
similar_movies = similarity_matrix[movie_data.index[0]].argsort()[::-1][1:10]
print(f"Film : {data_movies.loc[data_movies['movieId']==film_index][['movieId', 'title', 'genres']]}")

print("Rekomendasi:")
pd.DataFrame({
    "movieId" : data_movies.iloc[similar_movies]["movieId"],
    "title" : data_movies.iloc[similar_movies]["title"],
    "genres" : data_movies.iloc[similar_movies]["genres"],
}).head(-1)


 Spider-Man 3 (2007)


Film :        movieId                title                                 genres
11561    52722  Spider-Man 3 (2007)  action|adventure|sci-fi|thriller|imax
Rekomendasi:


,movieId,title,genres
46655,172891,Robotropolis (2011),action|adventure|sci-fi|thriller
5156,5264,Clockstoppers (2002),action|adventure|sci-fi|thriller
19422,101076,G.I. Joe: Retaliation (2013),action|adventure|sci-fi|thriller|imax
7931,8644,"I, Robot (2004)",action|adventure|sci-fi|thriller
29368,133759,Eyeborgs (2009),action|adventure|sci-fi|thriller
12619,61350,Babylon A.D. (2008),action|adventure|sci-fi|thriller
5241,5349,Spider-Man (2002),action|adventure|sci-fi|thriller
55109,191169,Absolute Zero (2005),action|adventure|sci-fi|thriller
